In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Data dir
raw_data_dir = "../data/mimic/raw/"
processed_data_dir = "../data/mimic/processed/"

# Read csv files
pat = pd.read_csv(raw_data_dir + "patients.csv")
adm = pd.read_csv(raw_data_dir + "admissions.csv")
proc = pd.read_csv(raw_data_dir + "procedures_icd.csv")
diag = pd.read_csv(raw_data_dir + "diagnoses_icd.csv")
ie = pd.read_csv(raw_data_dir + "icustays.csv")
d_icd_proc = pd.read_csv(raw_data_dir + "d_icd_procedures.csv")
d_icd_diag = pd.read_csv(raw_data_dir + "d_icd_diagnoses.csv")

#processed_mimic = pd.read_csv(processed_data_dir + "sequential_mimic.csv")

/share/apps/anaconda-ci/fgci-centos7-anaconda/software/anaconda/2021-03-tf2/e54ecce8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
adm["admittime"] = pd.to_datetime(adm["admittime"])
adm["dischtime"] = pd.to_datetime(adm["dischtime"])

diag['icd_version'] = diag['icd_version'].astype(int)
proc['icd_version'] = proc['icd_version'].astype(int)

# Inner join to get demographics
demo = pd.merge(pd.merge(pat,adm,on='subject_id'),ie,on=['subject_id', 'hadm_id'])

# Assign age, all patients are older than 18 years old
demo = demo.assign(admission_age = demo['admittime'].dt.year - demo['anchor_year'] + demo['anchor_age'])

# Merge with procedure and diagnosis
demo = pd.merge(demo, diag, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "diag_icd_code", "icd_version": "diag_icd_version", "seq_num": "diag_seq_num"})
demo = pd.merge(demo, proc, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "proc_icd_code", "icd_version": "proc_icd_version", "seq_num": "proc_seq_num"})

# Group into higher order categories
def group_and_prefix_diag(code,version):
    return str(version) + "_" + code[:3]

def group_and_prefix_proc(code,version):
    s =  "IP_" + str(version) + "_" + str(code)[:3]
    return ''.join(s.split())

demo['diag_icd_code'] = demo.apply(lambda x: group_and_prefix_diag(x.diag_icd_code, x.diag_icd_version), axis = 1)
demo['proc_icd_code'] = demo.apply(lambda x: group_and_prefix_proc(x.proc_icd_code, x.proc_icd_version), axis = 1)
demo

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,admittime,dischtime,deathtime,...,intime,outtime,los,admission_age,diag_seq_num,diag_icd_code,diag_icd_version,proc_seq_num,proc_icd_code,proc_icd_version
0,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065,30,4,9_296,9,5,IP_9_960,9
1,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065,30,4,9_296,9,4,IP_9_790,9
2,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065,30,4,9_296,9,3,IP_9_790,9
3,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065,30,4,9_296,9,2,IP_9_781,9
4,10003700,F,27,2162,2008 - 2010,NaN,28623837,2165-04-24 04:59:00,2165-04-26 22:34:00,NaN,...,2165-04-24 05:43:00,2165-04-24 09:13:20,0.146065,30,4,9_296,9,1,IP_9_781,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4782716,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-24 05:26:00,2161-08-30 23:48:04,6.765324,63,10,9_292,9,2,IP_9_967,9
4782717,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-24 05:26:00,2161-08-30 23:48:04,6.765324,63,6,9_530,9,1,IP_9_960,9
4782718,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-24 05:26:00,2161-08-30 23:48:04,6.765324,63,6,9_530,9,2,IP_9_967,9
4782719,19999068,M,63,2161,2011 - 2013,NaN,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,...,2161-08-24 05:26:00,2161-08-30 23:48:04,6.765324,63,4,9_920,9,1,IP_9_960,9


In [8]:
# Get LOS, remove outliers
demo_filter = demo[demo['los'] < 51]
los = demo_filter["los"].unique()

# Seperate into short/stay (long: los > 10 days)
demo_filter = demo_filter.assign(long_los = demo['los'] > 10)

# Filter out duplicates procedures per visit
demo_proc = demo_filter[['hadm_id', 'proc_icd_code', 'long_los']].drop_duplicates()
demo_diag = demo_filter[['hadm_id', 'diag_icd_code', 'long_los']].drop_duplicates()

In [9]:
print("Short los top 10 proc: \n", demo_proc[demo_proc['long_los'] == False]['proc_icd_code'].value_counts()[:10])
print("Long los top 10 proc: \n", demo_proc[demo_proc['long_los'] == True]['proc_icd_code'].value_counts()[:10])
print("Top 10 proc: \n", demo_proc['proc_icd_code'].value_counts()[:10])

#print("Short los top 10 diag: \n", demo_diag[demo_diag['long_los'] == False]['diag_icd_code'].value_counts()[:10])
#print("Long los top 10 diag: \n", demo_diag[demo_diag['long_los'] == True]['diag_icd_code'].value_counts()[:10])
#print("Top 10 diag: \n", demo_diag['diag_icd_code'].value_counts()[:10])
print(d_icd_proc[d_icd_proc['icd_code'] == 9671])

Short los top 10 proc: 
 IP_9_389     9222
IP_9_967     6755
IP_10_5A1    5924
IP_9_396     3977
IP_10_02H    3917
IP_9_960     3846
IP_9_966     3207
IP_9_885     3026
IP_9_372     2844
IP_9_361     2630
Name: proc_icd_code, dtype: int64
Long los top 10 proc: 
 IP_9_967     2196
IP_9_389     1953
IP_9_966     1909
IP_10_5A1    1341
IP_10_3E0    1283
IP_9_332     1260
IP_9_960     1211
IP_10_02H    1144
IP_10_0BH     820
IP_9_311      662
Name: proc_icd_code, dtype: int64
Top 10 proc: 
 IP_9_389     11175
IP_9_967      8951
IP_10_5A1     7265
IP_9_966      5116
IP_10_02H     5061
IP_9_960      5057
IP_9_396      4255
IP_10_3E0     3900
IP_9_885      3273
IP_9_332      3213
Name: proc_icd_code, dtype: int64
Empty DataFrame
Columns: [icd_code, icd_version, long_title]
Index: []


In [6]:
# Group codes into sequence
group_diag = demo.groupby(['subject_id','hadm_id'])['diag_icd_code'].apply(list)
group_proc = demo.groupby(['subject_id','hadm_id'])['proc_icd_code'].apply(list)
group_age = demo.groupby(['subject_id','hadm_id'])['admission_age'].apply(lambda x: list(x)[0])
group_gender = demo.groupby(['subject_id','hadm_id'])['gender'].apply(lambda x: list(x)[0])
group_ethnicity = demo.groupby(['subject_id','hadm_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_los = demo.groupby(['subject_id','hadm_id'])['los'].apply(lambda x: list(x)[0])

temp_final = pd.concat([group_age, group_gender,group_ethnicity, group_los, group_diag, group_proc], axis=1).reset_index()

In [13]:
# Again, group codes into sequence, separated by patient id
group_diag_0 = temp_final.groupby(['subject_id'])['diag_icd_code'].apply(list)
group_proc_0 = temp_final.groupby(['subject_id'])['proc_icd_code'].apply(list)
group_age_0 = temp_final.groupby(['subject_id'])['admission_age'].apply(lambda x: list(x))
group_gender_0 = temp_final.groupby(['subject_id'])['gender'].apply(lambda x: list(x)[0])
group_ethnicity_0 = temp_final.groupby(['subject_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_los_0 = temp_final.groupby(['subject_id'])['los'].apply(list)
group_hadm_id_0 = temp_final.groupby(['subject_id'])['hadm_id'].apply(list)

final = pd.concat([group_hadm_id_0, group_age_0, group_gender_0 ,group_ethnicity_0 , group_los_0, group_diag_0, group_proc_0], axis=1).reset_index()

In [34]:
# Sequentializes codes into sentences
def _sequentialize_codes(diag, proc):

    lst_len = len(diag)
    seqs = []
        
    for i in range(lst_len):
        
        # Filter unique values
        u_diag = list(set(diag[i]))
        u_proc = list(set(proc[i]))
        concat = u_diag + u_proc
        inner_seq = "<SEP>".join(concat)
        seqs.append(inner_seq)
    seq = ";".join(seqs)
    return seq

# Convert gender str to int
def _categorize_gender(gender):    
    return 0 if gender == "M" else 1

print(final['los'])
final['sequential_code'] = final.apply(lambda x: _sequentialize_codes(x.diag_icd_code, x.proc_icd_code), axis = 1)
final['gender'] = final.apply(lambda x: _categorize_gender(x.gender), axis = 1)

0                            [0.4975347222222222]
1        [1.1180324074074075, 0.9481134259259258]
2                             [1.338587962962963]
3                             [9.171817129629629]
4                            [1.3143518518518518]
                           ...                   
35097                         [6.765324074074075]
35098     [4.705752314814816, 1.5104166666666667]
35099                         [6.950370370370369]
35100                        [0.9607407407407408]
35101                        [1.9378472222222225]
Name: los, Length: 35102, dtype: object


In [35]:
# Assign treatment. Treatment is true if patient received 'IP_9_9671' procedure
def assign_treatment(proc):

    lst_len = len(proc)
    seqs = []
        
    for i in range(lst_len):
        if 'IP_9_967' in proc[i]:
            return 1
    return 0

final["treated"] = final.apply(lambda x: assign_treatment(x.proc_icd_code), axis = 1)

In [36]:
final.to_csv(processed_data_dir + "sequential_mimic.csv", index=False)

In [66]:
# Return max sequence length
def max_seq_length(seqs):
    
    # Flatten input list
    flatten = [item for sublist in seqs for item in sublist]
    
    # Split seq into token
    lens = []
    for s in flatten:
        
        # Strip white splace
        stripped_s = ''.join(s.split())
        
        # Split by ;
        stripped_s = stripped_s.split(";")
        
        for strip in stripped_s:
            # Split by <SEP>
            s = strip.split("<SEP>")
            lens.append(len(s))
            
    max_len = max(lens)
    return max_len

# Return max sequence length
def max_visit_length(seqs):
    
    # Flatten input list
    flatten = [item for sublist in seqs for item in sublist]
    
    # Split seq into token
    lens = []
    for s in flatten:
        
        # Strip white splace
        stripped_s = ''.join(s.split())
        
        # Split by ;
        stripped_s = stripped_s.split(";")
        
        for strip in stripped_s:
            
            lens.append(len(stripped_s))
    max_len = max(lens)
    return max_len

seqs = final.sequential_code.str[0:-1].str.split(',').tolist()

print("Max seq len:", max_seq_length(seqs))
print("Max visit len:", max_visit_length(seqs))
print("Treatment count:", final["treated"].value_counts())

Max seq len: 66
Max visit len: 20
Treatment count: 0    30666
1     4436
Name: treated, dtype: int64


In [39]:
def assign_old(age):
    return sum(age)/len(age) > 60

final["old_patient"] = final.apply(lambda x: assign_old(x.admission_age), axis=1)

In [43]:
old = final[final.old_patient == True]
young = final[final.old_patient == False]

print(young.treated.value_counts())
print(old.treated.value_counts())

0    10526
1     2170
Name: treated, dtype: int64
0    19292
1     3114
Name: treated, dtype: int64


In [45]:
2170/(10526+2170)

0.1709199747952111

In [44]:
3114/(19292 + 3114)

0.13898063018834242